# Music-Mood Classifier (CSI 4106 - Project - Group 29)
# Afrah Ali - 300049798 - aali179@uottawa.ca 
# Ribhav Khosla - 300087647 - rkhos052@uottawa.ca 
# Zain Malik - 300071476 - zmali081@uottawa.ca 

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

np.random.seed(42)

In [5]:
# load the dataset
df = pd.read_csv('dataset.csv')
print('Dataframe shape: ', df.shape)

# dropping non-features
dataset = df.drop(columns=['mood', 'track_name', 'artist', 'track_id'])
columns = list(dataset.columns)
print(columns)

print("Class distribution:")
print(df['mood'].value_counts())

#Label encoder
le = LabelEncoder()
df['mood_N'] = le.fit_transform(df['mood'])
df = df.drop(columns=['mood'])
df = df.rename(columns={'mood_N': 'mood'})
print(df)

# split dataset for training and testing
target = df['mood']
print(target)

# Set Training and Testing Data as 8:2
x_train, x_test, y_train, y_test = train_test_split(dataset, 
                                                    target, 
                                                    shuffle = True, 
                                                    test_size=0.2, 
                                                    random_state=1)

# Show the Training and Testing Data
print('Shape of training feature:', x_train.shape)
print('Shape of testing feature:', x_test.shape)
print('Shape of training label:', y_train.shape)
print('Shape of training label:', y_test.shape)

# Determine features with notable correlation
print("Features with notable correlation: ")
all_corr = dataset.corr()
for i in range(len(all_corr)):
    for j in range(i):
        if all_corr.iloc[i, j] > 0.5 or all_corr.iloc[i, j] < -0.5:
            print(str(all_corr.columns[i]) + " and " + str(all_corr.columns[j]) + " = " + str(all_corr.iloc[i, j]))

# Comparing energy values across moods
print("Mean energy for happy songs:", df.loc[df['mood'] == 'happy']['energy'].mean())
print("Mean energy for calm songs:", df.loc[df['mood'] == 'calm']['energy'].mean())
print("Mean energy for stressful songs:", df.loc[df['mood'] == 'stressful']['energy'].mean())
print("Mean energy for sad songs:", df.loc[df['mood'] == 'sad']['energy'].mean())

# Proportional class distribution
print("Class distribution of train set: ", y_train.value_counts())
print("Class distribution of test set: ", y_test.value_counts())


Dataframe shape:  (260, 16)
['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
Class distribution:
happy        65
sad          65
stressful    65
calm         65
Name: mood, dtype: int64
          track_name               artist                track_id  \
0        Upside Down         Jack Johnson  6shRGWCtBUOPFLFTTqXZIC   
1        Someone New               Hozier  0efT4YKQLQx2YHbp6vgRX8   
2       Little Talks  Of Monsters and Men  3a2tuvXCHbW5nuUckuHkKT   
3    Heart's Content       Brandi Carlile  0pegFWSUOTiG0sLVEfxtvA   
4     Sunday Morning             Maroon 5  4T5cqerbDXueYSVfXkIITo   
..               ...                  ...                     ...   
255    am ersten Tag       Hugo Vanbrooke  2gwhISMkdlhEqEP60P93Z1   
256    Amour naturel       Massimo Pavoni  39bh8hsTP2ZBQWH0E308rT   
257      Dawn Of Day          Sarah Seing  635M2GuMSoVunGBe7D7vWz   
258         Lumino

In [6]:
#Numerical pipeline for all data
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

# Sending data through pipeline to get fully transformed data
# 2D Numpy array
columns_transformed = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
train_data_transformed = pd.DataFrame(num_pipeline.fit_transform(x_train), columns=columns_transformed)
print(train_data_transformed)

# Creating pipelines for models
pipeline_svm = Pipeline([('scaler', StandardScaler())])
pipeline_dtc = Pipeline([('scaler', StandardScaler())])
pipeline_rfc = Pipeline([('scaler', StandardScaler())])
pipeline_gnb = Pipeline([('scaler', StandardScaler())])

     acousticness  danceability    energy  instrumentalness       key  \
0       -0.514931      1.492572  0.722318         -0.521550  0.840425   
1       -1.219636     -0.862821  1.798226          0.938070 -1.130040   
2        0.624283      0.145786 -1.325003         -0.513296 -0.567050   
3       -1.151568      0.869609  1.028061         -0.522125  0.840425   
4       -1.195381      0.181384  1.666640         -0.522150 -0.567050   
..            ...           ...       ...               ...       ...   
203     -1.204702      0.549228  1.763394         -0.522150  1.684910   
204      1.622533     -1.729035 -1.855216          2.741031 -0.567050   
205      1.490200     -1.645974 -0.740607          1.891319 -0.004060   
206     -0.799735      1.878216  0.335301         -0.522150 -0.004060   
207     -1.109279      1.130660  1.105465         -0.522150 -1.130040   

     liveness  loudness      mode  speechiness     tempo  time_signature  \
0   -0.338574  1.054692  0.814862    -0.462733 

In [7]:
# Support Vector Machine
svm = SVC()
pipeline_svm.steps.append(['model', svm])

# Decision Trees
dtc = DecisionTreeClassifier()
pipeline_dtc.steps.append(['model', dtc])

# Random Forests
rfc = RandomForestClassifier()
pipeline_rfc.steps.append(['model', rfc])

# Naive Bayes
gnb = GaussianNB()
pipeline_gnb.steps.append(['model', gnb])

Support Vector Machine

In [49]:
# Run GridSearchCV for support vector machine
gscv_svm = GridSearchCV(pipeline_svm, param_grid={'model__kernel':['linear', 'rbf'], 'model__C':[50,60,70,80,90,100,110,120,130,140], 'model__gamma':[ 0.01, 0.001, 0.0001,]})
# Setting the scoring metrics for the GridSearchCV
scoring = {"accuracy": "accuracy", "bal_accuracy": "balanced_accuracy", "F1_macro": "f1_macro"}
gscv_svm.set_params(scoring=scoring, refit='bal_accuracy')

# fitting data to models
gscv_svm.fit(train_data_transformed, y_train)

# best parameters
print("Support Vector Machine Best Parameters: ", gscv_svm.best_params_)

# best estimator
print("Support Vector Machine Best Estimator: ", gscv_svm.best_estimator_)

# best score
print("Support Vector Machine Best Score: ", gscv_svm.best_score_)

# Model Results
print("Support Vector Machine Results: ", gscv_svm.cv_results_)

Support Vector Machine Best Parameters:  {'model__C': 100, 'model__gamma': 0.0001, 'model__kernel': 'rbf'}
Support Vector Machine Best Estimator:  Pipeline(steps=[('scaler', StandardScaler()),
                ['model', SVC(C=100, gamma=0.0001)]])
Support Vector Machine Best Score:  0.6613636363636364
Support Vector Machine Results:  {'mean_fit_time': array([0.04367166, 0.00398736, 0.04548435, 0.00478139, 0.0438808 ,
       0.00358405, 0.05604134, 0.00461001, 0.05765619, 0.00381079,
       0.05724554, 0.00379028, 0.06363959, 0.00419693, 0.06343861,
       0.00377169, 0.06522288, 0.0043879 , 0.07940083, 0.00458183,
       0.07877951, 0.00397406, 0.07940431, 0.00398884, 0.07939534,
       0.00498977, 0.07838755, 0.00418887, 0.07718344, 0.00858111,
       0.10153589, 0.00479574, 0.09374819, 0.00479522, 0.08997407,
       0.00398912, 0.096947  , 0.0048068 , 0.09035821, 0.00424581,
       0.09108367, 0.0041995 , 0.10196209, 0.00497212, 0.11075649,
       0.00341935, 0.10165591, 0.00518665, 0

Decision Tree Classifier

In [50]:
# Run GridSearchCV for decision tree classifier
gscv_dtc = GridSearchCV(pipeline_dtc, param_grid={'model__max_depth':[1,2,3,4,5,6,7,8,9,10], 'model__min_samples_split':[25,35,45,55,65,75,85,95], 'model__min_samples_leaf':[10,15,20,25,30,35,40]})
# Setting the scoring metrics for the GridSearchCV
scoring = {"accuracy": "accuracy", "bal_accuracy": "balanced_accuracy", "F1_macro": "f1_macro"}
gscv_dtc.set_params(scoring=scoring, refit='bal_accuracy')

# fitting data to models
gscv_dtc.fit(train_data_transformed, y_train)

# best parameters
print("Decision Trees Best Parameters: ", gscv_dtc.best_params_)

# best estimator
print("Decision Trees Best Estimator: ", gscv_dtc.best_estimator_)

# best score
print("Decision Trees Best Score: ", gscv_dtc.best_score_)

# Model Results
print("Decision Trees Results: ", gscv_dtc.cv_results_)

Decision Trees Best Parameters:  {'model__max_depth': 5, 'model__min_samples_leaf': 15, 'model__min_samples_split': 25}
Decision Trees Best Estimator:  Pipeline(steps=[('scaler', StandardScaler()),
                ['model',
                 DecisionTreeClassifier(max_depth=5, min_samples_leaf=15,
                                        min_samples_split=25)]])
Decision Trees Best Score:  0.5304545454545455
Decision Trees Results:  {'mean_fit_time': array([0.00279374, 0.00279136, 0.00319176, 0.0027925 , 0.00239882,
       0.00298109, 0.00218773, 0.00259542, 0.00259957, 0.00199862,
       0.00298882, 0.00259299, 0.0029922 , 0.00258889, 0.00219831,
       0.00259266, 0.002601  , 0.00279398, 0.00200338, 0.00217843,
       0.00238981, 0.00221624, 0.00259919, 0.00258613, 0.0028069 ,
       0.00239344, 0.0027925 , 0.00259314, 0.00219364, 0.00259938,
       0.00299263, 0.0025928 , 0.00200534, 0.00238051, 0.00279894,
       0.00277653, 0.0024004 , 0.00240006, 0.00260577, 0.00220122,
       0.00

Random Forest Classifier

In [51]:
# Run GridSearchCV for random forest classifier
gscv_rfc = GridSearchCV(pipeline_rfc, param_grid={'model__n_estimators':[1,5,10,15,20,25,30,35,40], 'model__max_depth':[ 3,5,7,9,11,13,15,17,19,21,23,25,27,29], 'model__bootstrap':[True, False]})
# Setting the scoring metrics for the GridSearchCV
scoring = {"accuracy": "accuracy", "bal_accuracy": "balanced_accuracy", "F1_macro": "f1_macro"}
gscv_rfc.set_params(scoring=scoring, refit='bal_accuracy')

# fitting data to models
gscv_rfc.fit(train_data_transformed, y_train)

# best parameters
print("Random Forests Best Parameters: ", gscv_rfc.best_params_)

# best estimator
print("Random Forests Best Estimator: ", gscv_rfc.best_estimator_)

# best score
print("Random Forests Best Score: ", gscv_rfc.best_score_)

# Model Results
print("Random Forests Results: ", gscv_rfc.cv_results_)

Random Forests Best Parameters:  {'model__bootstrap': True, 'model__max_depth': 15, 'model__n_estimators': 40}
Random Forests Best Estimator:  Pipeline(steps=[('scaler', StandardScaler()),
                ['model',
                 RandomForestClassifier(max_depth=15, n_estimators=40)]])
Random Forests Best Score:  0.655909090909091
Random Forests Results:  {'mean_fit_time': array([0.00419683, 0.00736666, 0.01236229, 0.01714745, 0.02133722,
       0.02594395, 0.03071055, 0.03549871, 0.04209447, 0.00378423,
       0.00797729, 0.01279144, 0.0183579 , 0.02273288, 0.02752013,
       0.03311162, 0.03809729, 0.0426856 , 0.00399036, 0.00777683,
       0.01276407, 0.01815114, 0.02293868, 0.02812433, 0.03430772,
       0.0400949 , 0.04327083, 0.00378942, 0.00756636, 0.01336923,
       0.01815872, 0.0229382 , 0.02911553, 0.03350463, 0.03990436,
       0.04488487, 0.00358372, 0.00817714, 0.01296072, 0.01853604,
       0.02313828, 0.02853661, 0.03470564, 0.03889852, 0.04409142,
       0.0031981 , 

Naive Bayes

In [11]:
# Run GridSearchCV for naive bayes
gscv_gnb = GridSearchCV(pipeline_gnb, param_grid={})

# Setting the scoring metrics for the GridSearchCV
scoring = {"accuracy": "accuracy", "bal_accuracy": "balanced_accuracy", "F1_macro": "f1_macro"}
gscv_gnb.set_params(scoring=scoring, refit='bal_accuracy')

# fitting data to models
gscv_gnb.fit(train_data_transformed, y_train)

# best parameters
print("Naive Bayes Best Parameters: ", gscv_gnb.best_params_)

# best estimator
print("Naive Bayes Best Estimator: ", gscv_gnb.best_estimator_)

# best score
print("Naive Bayes Best Score: ", gscv_gnb.best_score_)

# Model Results
print("Naive Bayes Results: ", gscv_gnb.cv_results_)

Naive Bayes Best Parameters:  {}
Naive Bayes Best Estimator:  Pipeline(steps=[('scaler', StandardScaler()), ['model', GaussianNB()]])
Naive Bayes Best Score:  0.5522727272727271
Naive Bayes Results:  {'mean_fit_time': array([0.00418816]), 'std_fit_time': array([0.00116301]), 'mean_score_time': array([0.00259352]), 'std_score_time': array([0.00048872]), 'params': [{}], 'split0_test_accuracy': array([0.57142857]), 'split1_test_accuracy': array([0.54761905]), 'split2_test_accuracy': array([0.61904762]), 'split3_test_accuracy': array([0.46341463]), 'split4_test_accuracy': array([0.53658537]), 'mean_test_accuracy': array([0.54761905]), 'std_test_accuracy': array([0.0507564]), 'rank_test_accuracy': array([1]), 'split0_test_bal_accuracy': array([0.575]), 'split1_test_bal_accuracy': array([0.55454545]), 'split2_test_bal_accuracy': array([0.62727273]), 'split3_test_bal_accuracy': array([0.46818182]), 'split4_test_bal_accuracy': array([0.53636364]), 'mean_test_bal_accuracy': array([0.55227273]),

In [53]:
# test data prediction
test_data_transformed = pd.DataFrame(num_pipeline.fit_transform(x_test), columns=columns_transformed)
predict_labels_svm = gscv_svm.predict(test_data_transformed)
predict_labels_dtc = gscv_dtc.predict(test_data_transformed)
predict_labels_rfc = gscv_rfc.predict(test_data_transformed)
predict_labels_gnb = gscv_gnb.predict(test_data_transformed)

# classification report and the confusion matrix
print(classification_report(y_test, predict_labels_svm))
print(confusion_matrix(y_test, predict_labels_svm))

print(classification_report(y_test, predict_labels_dtc))
print(confusion_matrix(y_test, predict_labels_dtc))

print(classification_report(y_test, predict_labels_rfc))
print(confusion_matrix(y_test, predict_labels_rfc))

print(classification_report(y_test, predict_labels_gnb))
print(confusion_matrix(y_test, predict_labels_gnb))

              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.71      0.92      0.80        13
           2       0.87      0.93      0.90        14
           3       0.82      0.69      0.75        13

    accuracy                           0.79        52
   macro avg       0.79      0.78      0.78        52
weighted avg       0.79      0.79      0.78        52

[[ 7  2  2  1]
 [ 0 12  0  1]
 [ 0  1 13  0]
 [ 2  2  0  9]]
              precision    recall  f1-score   support

           0       0.53      0.67      0.59        12
           1       0.67      0.77      0.71        13
           2       0.82      0.64      0.72        14
           3       0.64      0.54      0.58        13

    accuracy                           0.65        52
   macro avg       0.66      0.65      0.65        52
weighted avg       0.67      0.65      0.66        52

[[ 8  2  2  0]
 [ 2 10  0  1]
 [ 2  0  9  3]
 [ 3  3  0  7]]
        

In [54]:

results = {
    'Model': ['SVM', 'Decision Tree', 'Random Forest Classifier', 'Naive Bayes'], 
    'Best Parameters (train)': [gscv_svm.best_params_, gscv_dtc.best_params_, gscv_rfc.best_params_, gscv_gnb.best_params_],
    'Best accuracy (train)':[max(gscv_svm.cv_results_['mean_test_accuracy']), max(gscv_dtc.cv_results_['mean_test_accuracy']), max(gscv_rfc.cv_results_['mean_test_accuracy']), max(gscv_gnb.cv_results_['mean_test_accuracy'])],
    'Best F1 Macro (train)': [max(gscv_svm.cv_results_['mean_test_F1_macro']), max(gscv_dtc.cv_results_['mean_test_F1_macro']), max(gscv_rfc.cv_results_['mean_test_F1_macro']), max(gscv_gnb.cv_results_['mean_test_F1_macro'])],
    'Best Accuracy (test)': [accuracy_score(y_test, predict_labels_svm), accuracy_score(y_test, predict_labels_dtc), accuracy_score(y_test, predict_labels_rfc), accuracy_score(y_test, predict_labels_gnb)],
    'Best F1 Macro (test)': [f1_score(y_test, predict_labels_svm, average='macro'), f1_score(y_test, predict_labels_dtc, average='macro'), f1_score(y_test, predict_labels_rfc, average='macro'), f1_score(y_test, predict_labels_gnb, average='macro')]
}

table = pd.DataFrame(results)
print(table)

                      Model  \
0                       SVM   
1             Decision Tree   
2  Random Forest Classifier   
3               Naive Bayes   

                             Best Parameters (train)  Best accuracy (train)  \
0  {'model__C': 100, 'model__gamma': 0.0001, 'mod...               0.658653   
1  {'model__max_depth': 5, 'model__min_samples_le...               0.533101   
2  {'model__bootstrap': True, 'model__max_depth':...               0.654007   
3                                                 {}               0.547619   

   Best F1 Macro (train)  Best Accuracy (test)  Best F1 Macro (test)  
0               0.643830              0.788462              0.778305  
1               0.517571              0.653846              0.652553  
2               0.648286              0.692308              0.674912  
3               0.509743              0.711538              0.709358  
